# Scientific Programming: A Crash Course

## Bonus Class 3 – High Performance Computing

In science we often need to write code that is very computationally intensive (e.g. simulating climate systems or processing huge fMRI datasets). Some computational problems are mainly limited by the CPU – the speed at which the computer can perform computational operations. Other problems are limited by the amount of memory available – the amount of information the computer can represent at the same time. In this class, we'll focus on the former – speed – and look at some practical strategies for optimizing your code. First, however, it's useful to have a theoretical framework in mind for thinking about the speed of programs.

## Time Complexity

In computer science, **time complexity** is a way of measuring how much time an algorithm takes to run in relation to the number of inputs it receives. For example, a simple function that involves just a single for-loop might have **linear time complexity**; if you double the number of items, it will take double the amount of time to process them all. The time complexity of a program is often expressed using "**big-O**" notation, a mathematical description of how the runtime relates to the number of inputs *n*. Here are some common time complexities, ordered from best to worst:

- $O(1)$ **Constant time** The program always takes the same amount of time, regardless of the number of inputs.
- $O(\log n)$ **Logarithmic time** The runtime scales with the logarithm of the number of inputs.
- $O(n)$ **Linear time** The runtime scales in direct proportion to the number of inputs.
- $O(n^2)$ **Quadratic time** The runtime scales with the square of the number of inputs.
- $O(2^n)$ **Exponential time** The runtime scales exponentially with respect to the number of inputs.

If you've written a function with factorial time complexity – $O(n!)$ – then the time it takes to run will grow extremely quickly with the number of items. If it takes one second to process a single item. Then, 3 items would take 6 seconds (not too bad); 6 items would take 12 minutes (okay, I can wait); 9 items would take four days (OMG); and 12 items would take 15 years (too late, I died already!). Factorial time complexity is really bad.

However, in day-to-day programming you don't usually need to worry about analyzing the time complexity of your code. In any case, modern computers are super fast, so there's no point wasting lots of time analyzing and optimizing your code just to make it run a few seconds faster. Nevertheless, as you scale-up to bigger datasets or more complex analyses, you might start to bump into hard limits that necessitate smarter algorithms or greater compute resources.

## Strategy 1: Vectorization

Often the most straight-forward strategy to speed up some scientific code is to take advantage of the **vectorized operations** in packages like NumPy. For example, the following list comprehension (i.e. just using standard Python code) calculates the first 10 million square numbers, and, on my computer, it takes around three seconds to run:

In [ ]:
squares = [x**2 for x in range(10_000_000)]

Now compare the same thing using NumPy:

In [ ]:
import numpy as np

squares = np.arange(10_000_000) ** 2

For me it's pretty much instantaneous. This is because operations on NumPy arrays are highly optimized and efficient. If possible, and when efficiency really matters, you should try to write your code in pure NumPy without mixing in too much standard Python. In some cases, not only do you get sweet efficiency gains, but you also end up with code that is simpler and more readable too.

## Strategy 2: Code Optimization

There are usually many different ways to approach a given computational problem, some of which may have better time complexity than others. To demonstrate, let's reconsider the `two_birthdays_are_same()` function from the end of Class 4. Recall that the function takes a list of birthdays – basically just a list of numbers – and returns `True` or `False`; `True` if any two of the numbers are the same or `False` if they are all different from each other. I've deliberately designed this version of the function to be inefficient. Before reading on, see if you can spot some ways to make it better.

In [ ]:
def two_birthdays_are_same(birthdays):
    two_same = False
    for i in range(len(birthdays)):
        for j in range(len(birthdays)):
            if i == j:
                continue
            if birthdays[i] == birthdays[j]:
                two_same = True
    return two_same


two_birthdays_are_same([ 68, 212, 363, 135, 32, 162, 326, 68 ])

This version of the `two_birthdays_are_same()` function has quadratic time complexity. I know this because there is one loop that runs *n* times nested inside another loop that runs *n* times; thus, the chunk of code in the middle – the if-statements – will get run $n \times n$, or $n^2$, times. For eight input numbers – like in the example above – the function needs to perform 64 comparisons. But we can easily improve on this (and simplify the code in the process) by thinking carefully about what the algorithm actually *needs* to do. At the moment, the function compares each number to every other number, but there's a lot of wasted effort here. If I've already compared the first number to the second number, I don't also need to compare the second number to the first number; that's just wasting time. Here's a new version of the code that eliminates this redundancy:

In [ ]:
def two_birthdays_are_same(birthdays):
    two_same = False
    for i in range(len(birthdays)):
        for j in range(i + 1, len(birthdays)):
            if birthdays[i] == birthdays[j]:
                two_same = True
    return two_same


two_birthdays_are_same([ 68, 212, 363, 135, 32, 162, 326, 68 ])

In this version, the inner, nested loop starts iteration from `i + 1`, which has the effect of only considering the unique pairs of numbers in the input list. Specifically, the first number is compared to the second, third, fourth...; the second number is compared to the third, fourth, fifth...; the third number is compared to the fourth, fifth, sixth...; and so on. So now, the runtime scales in proportion to the number of ways you can pair *n* items, or, more formally, its time complexity is $O(\frac{n^2 - n}{2})$, which is better than quadratic time. 

But it's possible to do even better. Here's another implementation of the function:

In [ ]:
def two_birthdays_are_same(birthdays):
    for i in range(len(birthdays)):
        for j in range(i + 1, len(birthdays)):
            if birthdays[i] == birthdays[j]:
                return True
    return False


two_birthdays_are_same([ 68, 212, 363, 135, 32, 162, 326, 68 ])

In this version, we are taking advantage of another shortcut. As soon as we find two numbers that match, we can immediately suspend the function and `return True`; we don't need to look at the rest of the numbers because we already know that the list contains at least one matching pair – and that's all that matters for the purpose of this particular function. In terms of the Python code, we have removed the temporary variable `two_same` and just inserted the return-statement at the point where were find a matching pair. If the function gets all the way to the end without finding any matches, then it returns `False`. The important thing to note here is that, as soon as you hit a return-statement, the function immediately exits, which includes exiting any loops you happen to be inside. This is a type of **short-circuiting** – a computational shortcut that is possible when you can logically deduce the outcome.

In this case it's a little tricky to describe the time complexity in terms of an equation. On average, it will be better than $O(\frac{n^2 - n}{2})$, but in the worst case scenario – that is, when none of the numbers are the same – it will still require $\frac{n^2 - n}{2}$ comparisons. Thus, computer scientists often distinguish between average time complexity and worst-case time complexity.

## Strategy 3: Just-In-Time (JIT) Compilation

In some languages, such as C, the programs you write are not run directly; instead, there is a two-step process of first **compiling** the code and then running it. Python is an interpreted language, meaning there is no separate compile step required – the code is interpreted directly. (Technically, your Python code is compiled to "bytecode", but this is an invisible step that you don't have to worry about, so to all intents and purposes, Python can be considered a non-compiled language.)

Generally speaking, compiled languages, like C, are much faster than interpreted languages, like Python. This is because the compile step translates the high-level code into instructions that can be directly run on the computer's hardware, and the compiler will also try to optimize the code to maximize efficiency. Nevertheless, the advantage of interpreted languages, like Python, is that they are generally easier to write and play around with: You can quickly and easily sketch out some ideas without any annoying compile step. Furthermore, when writing in a compiled language, like C, you have to write your code in a more explicit way to help the compiler understand it fully.

However, there is also an intermediate option: **Just-In-Time (JIT) compilation**. Roughly speaking, JIT compilation allows you to use the language as if were an interpreted language, while also taking advantage of the speed benefits of compilation: Important parts of the code get compiled "just-in-time" (i.e. certain bits of code are automatically compiled if and when they are needed).

There are a few different options for doing JIT compilation in Python, but one very nice option is the [Numba](https://numba.pydata.org) package. Numba makes it super easier to take advantage of JIT compilation with almost no effort! To try it out, you will need to make sure Numba is installed. If you're using Anaconda, it will probably be installed already; otherwise, check the [installation instructions here](https://numba.readthedocs.io/en/stable/user/installing.html). If it's correctly installed, you should be able to run the following line to import Numba:

In [ ]:
from numba import jit

Here's the birthday paradox code that we worked on in the previous class – the only difference is that I've increased the number of simulations to 10,000 (it used to be 1000). Try running the code and time how long it takes to run on your computer (for me it takes about 32 seconds).

In [ ]:
import numpy as np

def generate_birthdays(n_people):
    return np.random.randint(1, 366, n_people)

def two_birthdays_are_same(birthdays):
    for i in range(len(birthdays)):
        for j in range(i + 1, len(birthdays)):
            if birthdays[i] == birthdays[j]:
                return True
    return False

def probability_of_shared_bithday(n_people, n_sims=10000):
    count = 0
    for _ in range(n_sims):
        room_full_of_people = generate_birthdays(n_people)
        count += two_birthdays_are_same(room_full_of_people)
    return count / n_sims

probs = [ probability_of_shared_bithday(n) for n in range(2, 101) ]
print(probs)

Now here's the same code again, but this time I've added JIT compilation. Specifically, on the first line I imported Numba, and then above each function I've added `@jit`. Try running the code again to see how long it takes now. (N.b. the first time you run the code it may take a few moments because the functions get compiled the first time you run them, but subsequent runs should be faster).

In [ ]:
from numba import jit
import numpy as np

@jit
def generate_birthdays(n_people):
    return np.random.randint(1, 366, n_people)

@jit
def two_birthdays_are_same(birthdays):
    for i in range(len(birthdays)):
        for j in range(i + 1, len(birthdays)):
            if birthdays[i] == birthdays[j]:
                return True
    return False

@jit
def probability_of_shared_bithday(n_people, n_sims=10000):
    count = 0
    for _ in range(n_sims):
        room_full_of_people = generate_birthdays(n_people)
        count += two_birthdays_are_same(room_full_of_people)
    return count / n_sims

probs = [ probability_of_shared_bithday(n) for n in range(2, 101) ]
print(probs)

How much faster is the code? For me it takes about one second, so we've just achieved a 32× speedup with almost no effort. Nice! In fact, it's so fast that I can even increase the number of simulations to 100,000 and it still only takes about five seconds, implying a 64× speedup.

I wouldn't recommend that you automatically add `@jit` to every function you write because Numba is not able to JIT-compile all functions; it works best on relatively simple code that only uses basic data types and core NumPy functions. However, if you are able to identify and target the main bottlenecks in your code, Numba can get you some pretty decent gains.

(By the way, in case you're wondering what the `@` does, these are called [decorators](https://www.geeksforgeeks.org/decorators-in-python/).)

## Strategy 4: Parallelization

If you've tried all of the above and you still need more speed, the next port of call is probably to parallelize your code. Python is single-threaded, which essentially means it can only perform one operation at a time. However, most modern computers have multiple "cores" (my computer has eight) that can perform computational operations independently and in parallel. Thus, for certain types of problem, you can potentially make things go seveal times faster by running things in parallel on different cores. However, not all computational problems can easily be handled this way. In particular, if you have a situation where one piece of code relies on another piece of code (e.g. the input to one function is the output of another function), then these two pieces of code cannot, by definition, be run independently.

The [Multiprocessing](https://docs.python.org/3/library/multiprocessing.html) module within the Python Standard Library provides lots of options for running function calls in parallel. I've provided an example below, although note that this code cannot be run inside a notebook; it must be run as a Python script because the Multiprocessing module has to spawn other processes (try using Spyder or another IDE/editor if you want to try it out). Essentially, I took advantage of the fact that each room size is an independent computational problem that can be run independently of all the others. For me, this version of the code takes about six seconds to run, compared to 32 seconds for the non-parallelized version, so about five times faster.

```python
from multiprocessing import Pool
import numpy as np

def generate_birthdays(n_people):
    return np.random.randint(1, 366, n_people)

def two_birthdays_are_same(birthdays):
    for i in range(len(birthdays)):
        for j in range(i + 1, len(birthdays)):
            if birthdays[i] == birthdays[j]:
                return True
    return False

def probability_of_shared_bithday(n_people, n_sims=10000):
    count = 0
    for _ in range(n_sims):
        room_full_of_people = generate_birthdays(n_people)
        count += two_birthdays_are_same(room_full_of_people)
    return count / n_sims

if __name__ == '__main__':
    
    N_CORES = 8 # insert number of cores here

    with Pool(N_CORES) as pool:
        probs = pool.map(probability_of_shared_bithday, range(2, 101))

    print(probs)
```

Again, I wouldn't recommend that you just parallelize all your code by default. Parallelization adds a certain amount of computational overhead that can actually make things slower, and parallelized code tends to be harder to understand and prone to bugs (notably [race conditions](https://en.wikipedia.org/wiki/Race_condition)).

## Strategy 5: HPC Cluster

Finally, if all else fails, just throw a lot of compute resources at the problem. SISSA has a high performance compute cluster called [Ulysses](https://www.itcs.sissa.it/services/computing/hpc?s[]=ulysses). Using it is a bit fiddly and I believe you need to request access, but I'll try to give you the general idea here about how it works.

First, you log into the server using SSH; from the terminal, you'll type a command like this:

```shell
ssh SISSA_USERNAME@frontend2.hpc.sissa.it 
```

On the server, create or upload your Python script, let's call it `birthday_paradox.py`:

```python
from sys import argv
import numpy as np

def generate_birthdays(n_people):
    return np.random.randint(1, 366, n_people)

def two_birthdays_are_same(birthdays):
    for i in range(len(birthdays)):
        for j in range(i + 1, len(birthdays)):
            if birthdays[i] == birthdays[j]:
                return True
    return False

def probability_of_shared_bithday(n_people, n_sims=10000):
    count = 0
    for _ in range(n_sims):
        room_full_of_people = generate_birthdays(n_people)
        count += two_birthdays_are_same(room_full_of_people)
    return count / n_sims

if __name__ == '__main__':

    if len(argv) == 2:
        n_people = int(argv[1])
        prob = probability_of_shared_bithday(n_people)
        print(prob)
```

Here I've designed the script to take an input number from the command line, which determines the room size, This allows you to run a command like `python birthday_paradox.py 50` to run the simulation for a room size of 50.

Next you create a shell script, let's call it `birthday_paradox.sh`. This file describes what resources you are requesting (how much compute time, number of cores, and RAM) and what program needs to be executed (i.e. the Python script that we just made). The shell script will look something like this:

```bash
#!/bin/bash

#SBATCH -N1
#SBATCH -n1
#SBATCH --time=0:02:00
#SBATCH --mem=1G

module load python3/3.8
python birthday_paradox.py $SLURM_ARRAY_TASK_ID
```

And finally, you submit this shell script to one of the compute nodes using the `sbatch` command:

```shell
sbatch --array=2-100 -p regular2 birthday_paradox.sh
```

Here I've specified an "array-job" with the range of numbers 2...100. The idea is that we will run each room size (the smallest being 2 and the largest being 100) on a separate compute core. So this command will send the shell script to 98 separate cores, each with an individual task ID ranging from 2 to 100. For each of these separate runs, the relevant task ID will be inserted into the shell script where it says `$SLURM_ARRAY_TASK_ID`, and this in turn will be passed to the Python script, which will then calculate the probability for the relevant room size. Currently the Python code simply prints the result, which will get written out to a log file, but probably you'd want to write the estimated probability to a file so that you can later collate all the numbers produced by each independent run of the program.

As you can see, getting everything set up correctly is quite fiddly, so I'd only recommend this option once you've exhausted all others. Nevertheless, it's especially useful if you have long-running code that can easily be chunked into discrete parts, and the great thing is you can free up your own computer for other day-to-day activities.

## Beware of Premature Optimization!

Lastly, a word of warning: There is little point in wasting lots of time optimizing your code just to squeeze out a few extra seconds. The amount of time you spend refactoring your code may outweigh any practical performance benefits and you may even introduce new bugs. Furthermore, if you do set out to optimize your code, be sure to do so in an evidence-based way. Use a [profiler](https://docs.python.org/3/library/profile.html) to find out how much time the program spends inside each function, and then target your optimizations on the main bottlenecks.